In [1]:
from gensim.models.word2vec import Word2Vec

In [15]:
import re
from sklearn import feature_extraction 
stop_words = feature_extraction.text.ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def preprocess(text):
  text = re.sub(r'[^\w\s]', '', str(text)) #remove punctuations
  text = re.sub(r'\d+', '', str(text)) #remove numbers
  text = text.lower() #lowercase
  text = " ".join(text.split()) #stripWhitespace
  text = text.split()
  text = [x for x in text if x not in stop_words] #remove stopwords
  text = [x for x in text if x not in ["elden", "ring"]] #remove task specific stopwords
  text = " ".join(text)
  # stemmer_ps = PorterStemmer()  
  # text = [stemmer_ps.stem(word) for word in text.split()] #stemming
  # text = " ".join(text)
  # lemmatizer = WordNetLemmatizer()
  # text = [lemmatizer.lemmatize(word) for word in text.split()]  #lemmatization
  # text = " ".join(text)
  return(text)

In [3]:
import pandas as pd
data = pd.read_csv('elden_ring_steam_reviews.csv')

In [4]:
data.head(5)

,id,language,review,created,voted_up,votes_up,comment_count,steam_purchase,recieved_for_free,written_during_early_access,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played
0,134238864,english,being killed over and over again is fun,2023-03-07 14:19:11,True,0,0,False,False,False,47,2,355,355,346,2023-03-07 14:29:21
1,134238208,english,I write this review as I have 100% completed E...,2023-03-07 14:03:24,True,0,0,True,False,False,110,2,24238,5823,24238,2023-03-07 02:08:22
2,134238033,english,Fun,2023-03-07 13:59:13,True,0,0,True,False,False,11,1,912,912,881,2023-03-07 14:30:07
3,134237508,english,pretty cool.,2023-03-07 13:46:37,True,0,0,True,False,False,94,8,8497,2914,8497,2023-03-07 13:44:52
4,134236511,english,AMAZINGGGGGGGGGGGGG,2023-03-07 13:20:04,True,0,0,True,False,False,4,1,985,163,923,2023-03-07 14:26:41


In [31]:
data = data[data.language == "english"]

In [32]:
data['review_processed']=data['review'].apply(lambda x:preprocess(x))
data['review_processed']=data['review_processed'].apply(lambda x:x.split())

In [33]:
data

,id,language,review,created,voted_up,votes_up,comment_count,steam_purchase,recieved_for_free,written_during_early_access,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,review_processed
0,134238864,english,being killed over and over again is fun,2023-03-07 14:19:11,True,0,0,False,False,False,47,2,355,355,346,2023-03-07 14:29:21,"[killed, fun]"
1,134238208,english,I write this review as I have 100% completed E...,2023-03-07 14:03:24,True,0,0,True,False,False,110,2,24238,5823,24238,2023-03-07 02:08:22,"[write, review, completed, despite, new, thing..."
2,134238033,english,Fun,2023-03-07 13:59:13,True,0,0,True,False,False,11,1,912,912,881,2023-03-07 14:30:07,[fun]
3,134237508,english,pretty cool.,2023-03-07 13:46:37,True,0,0,True,False,False,94,8,8497,2914,8497,2023-03-07 13:44:52,"[pretty, cool]"
4,134236511,english,AMAZINGGGGGGGGGGGGG,2023-03-07 13:20:04,True,0,0,True,False,False,4,1,985,163,923,2023-03-07 14:26:41,[amazinggggggggggggg]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9789,131803069,english,Best platformer of the year.,2023-01-29 14:50:25,True,0,0,True,False,False,112,8,12095,1751,7565,2023-03-06 13:54:32,"[best, platformer, year]"
9790,131802918,english,"truly fun...very recommended, worth the money ...",2023-01-29 14:47:55,True,0,0,True,False,False,17,4,9467,39,8394,2023-02-26 17:51:52,"[truly, funvery, recommended, worth, money, time]"
9791,131802717,english,mimic is the friend we made along the way,2023-01-29 14:44:41,True,0,0,True,False,False,34,3,8098,6,5655,2023-02-23 10:17:41,"[mimic, friend, way]"
9792,131802309,english,"The lands of Elden Ring are vast and wild, fil...",2023-01-29 14:38:38,True,1,0,True,False,False,70,8,9472,201,9191,2023-03-05 13:39:17,"[lands, vast, wild, filled, dangers, lurking, ..."


In [107]:
model = Word2Vec(sentences=data['review_processed'].tolist(), vector_size=300,sg=1,min_count=5,window=10,workers=50,seed=10,epochs=100)
#vector_size (int, optional) – Dimensionality of the word vectors.
#min_count (int, optional) – Ignores all words with total frequency lower than this.
#window (int, optional) – Maximum distance between the current and predicted word within a sentence.
#workers (int, optional) – Use these many worker threads to train the model (=faster training with multicore machines).
#sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
#epochs (int, optional) – Number of iterations (epochs) over the corpus. (Formerly: iter)

#The meaning of most of the parameters are beyond the scope of this class. If interested, please check the official documentations: https://radimrehurek.com/gensim/models/word2vec.html

In [120]:
model.wv.most_similar('killed', topn=10)

[('exploit', 0.33646878600120544),
 ('elevator', 0.29332464933395386),
 ('rats', 0.2752256393432617),
 ('dragon', 0.2722066044807434),
 ('ruins', 0.26664888858795166),
 ('respawning', 0.2624953091144562),
 ('rat', 0.2578144371509552),
 ('respawn', 0.2554304301738739),
 ('grace', 0.2504226565361023),
 ('attempts', 0.24889318645000458)]

In [152]:
model.wv.most_similar('plot', topn=10)

[('gather', 0.30340901017189026),
 ('grasp', 0.3000735640525818),
 ('entertained', 0.29185399413108826),
 ('repeating', 0.29004380106925964),
 ('themes', 0.2753866910934448),
 ('noice', 0.27247852087020874),
 ('tone', 0.27215951681137085),
 ('consoles', 0.2704331576824188),
 ('typical', 0.2691148519515991),
 ('stellar', 0.26908132433891296)]

In [123]:
model.wv.most_similar('money', topn=10)

[('worth', 0.3450014591217041),
 ('gang', 0.3420043885707855),
 ('dollars', 0.3371058404445648),
 ('infinity', 0.3294423818588257),
 ('spare', 0.3290877342224121),
 ('ark', 0.32248052954673767),
 ('fortnite', 0.3201828896999359),
 ('deaf', 0.30601823329925537),
 ('burn', 0.2964648902416229),
 ('evolved', 0.28911903500556946)]

In [138]:
model.wv.most_similar('scene', topn=10)

[('cutting', 0.4096072018146515),
 ('scenes', 0.3821605443954468),
 ('disappoint', 0.36313316226005554),
 ('noice', 0.35858845710754395),
 ('marika', 0.35744160413742065),
 ('occasional', 0.3539219796657562),
 ('movie', 0.34203866124153137),
 ('spell', 0.3334980010986328),
 ('ight', 0.3309401571750641),
 ('social', 0.3287423253059387)]

In [149]:
model.wv.most_similar('achievement', topn=10)

[('discovering', 0.32832229137420654),
 ('ending', 0.30452629923820496),
 ('alas', 0.29158511757850647),
 ('relief', 0.28847309947013855),
 ('platinum', 0.28551721572875977),
 ('disappoint', 0.27296656370162964),
 ('frenzied', 0.27096638083457947),
 ('neat', 0.270506888628006),
 ('roaming', 0.2684386372566223),
 ('dogshit', 0.2679654657840729)]

In [160]:
model.wv.most_similar('music', topn=10)

[('spectacular', 0.345890611410141),
 ('seamlessly', 0.3309445083141327),
 ('ost', 0.3130759000778198),
 ('exception', 0.2837921679019928),
 ('beatiful', 0.2778931260108948),
 ('unusual', 0.2770550549030304),
 ('cutting', 0.26338258385658264),
 ('replayability', 0.26288795471191406),
 ('toptier', 0.2606656849384308),
 ('beats', 0.2584938406944275)]

In [161]:
model.wv.most_similar('surprise', topn=10)

[('humans', 0.36773577332496643),
 ('company', 0.3245186507701874),
 ('purchased', 0.32205909490585327),
 ('moon', 0.31840193271636963),
 ('beautifully', 0.31718453764915466),
 ('syndrome', 0.30310356616973877),
 ('addiction', 0.2997865378856659),
 ('pleasant', 0.2981395423412323),
 ('veri', 0.2937401831150055),
 ('reminds', 0.29072442650794983)]

In [165]:
model.wv.most_similar('ending', topn=10)

[('frenzied', 0.38064053654670715),
 ('endings', 0.3343939483165741),
 ('rannis', 0.3159030079841614),
 ('hug', 0.3082789480686188),
 ('azur', 0.30606716871261597),
 ('achievement', 0.30452626943588257),
 ('age', 0.2967379093170166),
 ('accomplished', 0.2804076671600342),
 ('wildly', 0.27869394421577454),
 ('completed', 0.27568483352661133)]

In [117]:
outdata=pd.DataFrame(model.wv.vectors)

In [118]:
outdata.to_csv('word2vec_ratemds.tsv',sep='\t',index=False,header=False)

In [119]:
pd.DataFrame(model.wv.index_to_key).to_csv('word2vec_ratemds_words.tsv',sep='\t',index=False,header=False)